In [2]:
import random
from glob import glob
import random
from tqdm import tqdm
import numpy as np
import pymysql
import orjson as json
import re

def create_db(database):
    db = pymysql.connect(host='localhost',
                         user='root',
                         password='123456',
                         database=database)
    return db


In [3]:
pixiv_db = create_db('pixiv_data')
sd_db = create_db("stable_diffusion")

sql = 'select id,name from tags'
cursor = sd_db.cursor()
cursor.execute(sql)
result = cursor.fetchall()
id_tag = {id:name for id,name in result}
tag_id = {name:id for id,name in result}

In [5]:
# 将tags.txt写入表all_prompts_remove_nfsw
def write_tags_to_table(connection, cursor, filename):
    not_in_db = []
    with open(filename, 'r') as file:
        tags = file.read().split(",")

        for i,tag in enumerate(tqdm(tags)):
            tag = tag.strip().replace(" ","_")
            if tag in tag_id:

                id = tag_id[tag]

                cursor.execute("INSERT IGNORE INTO all_prompts_remove_nfsw (prompt_tag_id, tag) VALUES (%s, %s)", (id, tag))
            else:
                not_in_db.append(tag)

        connection.commit()
    print("Tags written to 'all_prompts_remove_nfsw' table successfully.")
    return not_in_db



connection = create_db("stable_diffusion")
if connection:
    cursor = connection.cursor()

    not_in_db = write_tags_to_table(connection, cursor, r"C:\sd\my_code\all_prompts_remove_nfsw.txt")

    # 关闭连接
    cursor.close()
    connection.close()
print(len(not_in_db))
print(not_in_db)

100%|██████████| 8538/8538 [00:01<00:00, 4312.00it/s]

Tags written to 'all_prompts_remove_nfsw' table successfully.
335
['bat_(animal)', 'torn_pantyhose', 'rabbit', 'object_through_head', 'torn_thighhighs', 'white_pantyhose', 'black_pantyhose', 'female_child', 'upper_teeth_only', 'meiko_(vocaloid)', 'avatar_(ff11)', 'black_thighhighs', 'black_socks', 'male_child', 'bikini_top_only', 'breasts_out', 'bridal_garter', 'traditional_youkai', 'breasts_squeezed_together', 'white_thighhighs', 'dyed_bangs', 'single_hair_bun', 'brown_pantyhose', 'tachibana_kanade', 'mithra_(ff11)', 'stretching', 'chain_leash', 'fishnet_pantyhose', 'white_socks', 'grey_pantyhose', 'checkered_clothes', 'chest_sarashi', 'striped_pantyhose', 'orange_pantyhose', 'name_connection', 'side-tie_bikini_bottom', 'avatar_(ff14)', 'boo_tao_(genshin_impact)', 'clumsy_nun_(diva)', 'black_one-piece_swimsuit', 'osaki_amana', 'red_pantyhose', 'akizuki_ryo', 'united_states', 'pink_pantyhose', 'magazine_(object)', "holding_another's_wrist", 'legwear_garter', 'eyebrows_hidden_by_hair', 

In [54]:
#获取id和tag的对应关系
sql = 'select prompt_tag_id,tag from all_prompts_remove_nfsw'
cursor = sd_db.cursor()
cursor.execute(sql)
result = cursor.fetchall()
id_tag = {str(id):name for id,name in result}
tag_id = {name:str(id) for id,name in result}

In [55]:
#获取全部tags
sql = 'select tags,pixiv_id from entries'
cursor = sd_db.cursor()
cursor.execute(sql)
result = cursor.fetchall()


In [56]:
new_result = []
for tags,pixiv_id in tqdm(result):
    tags_id = tags
    try:

        tags_id = tags_id.replace('"',"").replace("[","").replace("]","").split(", ")
    except:
        print()
    if not isinstance(tags_id,list):
        print()
    new_tags_id = []
    for id in tags_id:

        id = id.replace(",","")

        if id in id_tag:
            new_tags_id.append(id)

    new_result.append(new_tags_id)




100%|██████████| 4790576/4790576 [00:59<00:00, 80762.95it/s] 


In [57]:

len_all_taggers = len(new_result)
train_num = int(len_all_taggers * 0.9)
test_num = int(len_all_taggers * 0.1)
train_output_json = {"id_tag":id_tag,"tags":new_result[:train_num]}
test_output_json = {"id_tag":id_tag,"tags":new_result[train_num:]}


with open("./data/bert/train.json","wb") as f:
    f.write(json.dumps(train_output_json)+b"\n")

with open("./data/bert/test.json","wb") as f:
    f.write(json.dumps(test_output_json) + b"\n")

